<a href="https://colab.research.google.com/github/flaviopradoaquino/TCC_MBA_USP_ESALQ/blob/main/TCC_USP_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação e Importação das Bibliotecas**

In [ ]:
!pip install -q yfinance

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import yfinance as yf
import datetime
from datetime import date, timedelta
from prophet.forecaster import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from scipy import stats

# **Construção do Dataset**

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')

In [ ]:
from datetime import datetime 
dt_atual = date.today()
td = timedelta(1095)
dt_final = datetime(2023, 3, 3)

In [ ]:
### Data de referência menos 1095 dias
df = yf.download("BOVA11.SA", start=(dt_final-td))['Close']

### Data atual menos 1095 dias
# df = yf.download("BOVA11.SA", start=(dt_atual-td))['Close']

### Intervalo de datas fixas
# df = yf.download("BOVA11.SA", start="2020-03-03", end="2023-03-03")['Close']

In [ ]:
df = df.reset_index()
df

In [ ]:
df_bova = pd.DataFrame(df['Close'])
df_bova

# **Excluindo Outliers**

In [ ]:
dados = df_bova
boxplot_bova = px.box(dados, y="Close")
boxplot_bova.show()

In [ ]:
q1 = np.quantile(dados, .25, method="midpoint")
q2 = np.quantile(dados, .5, method="midpoint")
q3 = np.quantile(dados, .75, method="midpoint")
IIQ = q3 - q1

In [ ]:
### Limite inferior do boxplot
limite_inferior = q1 - 1.5 * IIQ
limite_inferior

In [ ]:
### Limite superior do boxplot
limite_superior = q3 + 1.5 * IIQ
limite_superior

In [ ]:
df_bova_lim_inf = df_bova[df_bova['Close']>limite_inferior]
df_bova_clean = df_bova_lim_inf[df_bova_lim_inf['Close']<limite_superior]
df_bova = df_bova_clean
df_bova

# **Construindo o Modelo**

In [ ]:
### Normalização
df_normalizado = df_bova.copy()
for i in df_bova:
  df_normalizado[i] = df_bova[i] / df_bova[i][0]
df_normalizado

In [ ]:
### Calculando a taxa de retorno
df_taxa_retorno = np.log(1 + df_normalizado.pct_change())
df_taxa_retorno

# **Movimento Browniano com Drift**

In [ ]:
### Drift
drift = df_taxa_retorno.mean() - (0.5 * df_taxa_retorno.var())
drift

# **Retorno Diário**

In [ ]:
### Definição da quantidade de dias futuro e total de simulções
dias = 90
simulacao = 10

In [ ]:
### Desvio Padrão
desvio_padrao = df_taxa_retorno.std()
desvio_padrao

In [ ]:
### Padronização
Z = stats.norm.ppf(np.random.rand(dias, simulacao))

In [ ]:
Z.shape

In [ ]:
### Definição do retorno diário
retorno_diario = np.exp(drift.values + desvio_padrao.values * Z)

In [ ]:
retorno_diario.shape

In [ ]:
### Definição da matriz de zeros
previsao = np.zeros_like(retorno_diario)

In [ ]:
previsao[0] = df_bova.iloc[-1]
previsao

In [ ]:
### Gerando valores da previsão
for dia in range(1, dias):
  previsao[dia] = previsao[dia - 1] * retorno_diario[dia]

In [ ]:
previsao.shape

In [ ]:
previsao

# **Geração do Gráfico**

In [ ]:
### Matriz transposta
previsao.T

In [ ]:
### Parâmetros do gráfico
figura = px.line(title = 'Simulação de Monte Carlo - Ações da BOVA')
for i in range(len(previsao.T)):
  figura.add_scatter(y = previsao.T[i], name = i)
figura.show()

# **Avaliação da Previsão**

In [ ]:
from datetime import date, timedelta

dt_atual_02 = date.today()
td_02 = timedelta(129)

In [ ]:
df2 = yf.download("BOVA11.SA", start=(dt_atual_02-td_02))['Close']

In [ ]:
df2 = df2.reset_index()
df2

In [ ]:
df_bova_02 = pd.DataFrame(df2['Close'])
df_bova_02

In [ ]:
boxplot_avaliacao = px.box(df_bova_02, y="Close")
boxplot_avaliacao.show()

In [ ]:
len(df_bova_02)

In [ ]:
simulacao_01 = previsao.T[0][0:len(df_bova_02)]
simulacao_01

In [ ]:
len(simulacao_01)

In [ ]:
df_bova_02['Close']

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_bova_02['Close'], simulacao_01)

In [ ]:
erro = []
for i in range(len(previsao.T)):
  simulacao_01 = previsao.T[i][0:len(df_bova_02)]
  erro.append(mean_absolute_error(df_bova_02['Close'], simulacao_01))

In [ ]:
erro

In [ ]:
erro.index(max(erro)), erro.index(min(erro))

In [ ]:
max(erro),min(erro)

In [ ]:
figura = px.line(title = 'Previsões do preço das ações da BOVA - simulação')
figura.add_scatter(y = df_bova_02['Close'], name = 'Valor real')
figura.add_scatter(y = previsao.T[erro.index(max(erro))], name = 'Pior simulação')
figura.add_scatter(y = previsao.T[erro.index(min(erro))], name = 'Melhor simulação')